In [61]:
#Some Credits @campusX
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # In production you ahve to use a persistent storage like Redis or Postgres

d:\Agentic Ai\LangChain_GenAi_LangGraph\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load_dotenv()

# llm = ChatOpenAI()

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)

In [63]:
class JokeState(TypedDict):

    topic_of_joke: str
    actual_joke: str
    explanation: str

In [64]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic_of_joke"]}'
    response = llm.invoke(prompt).content

    return {'actual_joke': response}

In [65]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["actual_joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [66]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer1 = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer1)

In [67]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic_of_joke':'paid education'}, config=config1)

{'topic_of_joke': 'paid education',
 'actual_joke': 'Why did the student cross the road?  Because tuition was too high to afford a bridge.',
 'explanation': 'This is a play on words, using the common question "Why did the chicken cross the road?"  The expected answer is simple and straightforward (to get to the other side).  Instead, the joke offers a humorous, financially-motivated reason for crossing the road – implying that the student is so poor due to high tuition fees that they can\'t afford the more convenient (and presumably more expensive) option of using a bridge.  The humor comes from the unexpected, relatable, and slightly absurd reason given in the context of a well-known riddle.'}

In [68]:
workflow.get_state(config1)

StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student cross the road?  Because tuition was too high to afford a bridge.', 'explanation': 'This is a play on words, using the common question "Why did the chicken cross the road?"  The expected answer is simple and straightforward (to get to the other side).  Instead, the joke offers a humorous, financially-motivated reason for crossing the road – implying that the student is so poor due to high tuition fees that they can\'t afford the more convenient (and presumably more expensive) option of using a bridge.  The humor comes from the unexpected, relatable, and slightly absurd reason given in the context of a well-known riddle.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e8ac-f8e7-65a7-8002-21783b627fd7'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T12:31:50.129194+00:00', parent_config={'configur

In [69]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student cross the road?  Because tuition was too high to afford a bridge.', 'explanation': 'This is a play on words, using the common question "Why did the chicken cross the road?"  The expected answer is simple and straightforward (to get to the other side).  Instead, the joke offers a humorous, financially-motivated reason for crossing the road – implying that the student is so poor due to high tuition fees that they can\'t afford the more convenient (and presumably more expensive) option of using a bridge.  The humor comes from the unexpected, relatable, and slightly absurd reason given in the context of a well-known riddle.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e8ac-f8e7-65a7-8002-21783b627fd7'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T12:31:50.129194+00:00', parent_config={'configu

In [70]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic_of_joke':'unpaid_internships'}, config=config2)

{'topic_of_joke': 'unpaid_internships',
 'actual_joke': "Why did the unpaid intern bring a ladder to the office?  Because they heard the company was looking for someone to climb the corporate ladder... eventually, maybe, if they're lucky.",
 'explanation': 'The humor in the joke lies in the juxtaposition of the literal and figurative meanings of "climbing the corporate ladder."\n\n* **Literal:** The intern brings a ladder, a physical object used for climbing. This is a straightforward, almost absurdly literal interpretation of the idiom.\n\n* **Figurative:** "Climbing the corporate ladder" is an idiom meaning to advance one\'s career within a company, achieving higher positions and responsibilities.  This is the intended, understood meaning.\n\nThe joke\'s punchline highlights the intern\'s precarious position.  They\'re unpaid, implying a low status and little chance of advancement.  The phrase "eventually, maybe, if they\'re lucky" emphasizes the slim possibility of their actually ac

In [71]:
workflow.get_state(config2)

StateSnapshot(values={'topic_of_joke': 'unpaid_internships', 'actual_joke': "Why did the unpaid intern bring a ladder to the office?  Because they heard the company was looking for someone to climb the corporate ladder... eventually, maybe, if they're lucky.", 'explanation': 'The humor in the joke lies in the juxtaposition of the literal and figurative meanings of "climbing the corporate ladder."\n\n* **Literal:** The intern brings a ladder, a physical object used for climbing. This is a straightforward, almost absurdly literal interpretation of the idiom.\n\n* **Figurative:** "Climbing the corporate ladder" is an idiom meaning to advance one\'s career within a company, achieving higher positions and responsibilities.  This is the intended, understood meaning.\n\nThe joke\'s punchline highlights the intern\'s precarious position.  They\'re unpaid, implying a low status and little chance of advancement.  The phrase "eventually, maybe, if they\'re lucky" emphasizes the slim possibility o

In [72]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic_of_joke': 'unpaid_internships', 'actual_joke': "Why did the unpaid intern bring a ladder to the office?  Because they heard the company was looking for someone to climb the corporate ladder... eventually, maybe, if they're lucky.", 'explanation': 'The humor in the joke lies in the juxtaposition of the literal and figurative meanings of "climbing the corporate ladder."\n\n* **Literal:** The intern brings a ladder, a physical object used for climbing. This is a straightforward, almost absurdly literal interpretation of the idiom.\n\n* **Figurative:** "Climbing the corporate ladder" is an idiom meaning to advance one\'s career within a company, achieving higher positions and responsibilities.  This is the intended, understood meaning.\n\nThe joke\'s punchline highlights the intern\'s precarious position.  They\'re unpaid, implying a low status and little chance of advancement.  The phrase "eventually, maybe, if they\'re lucky" emphasizes the slim possibility 

### Time Travel

In [75]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f07e8ac-f965-634e-8000-5e88606453a5"}})

StateSnapshot(values={'topic_of_joke': 'unpaid_internships'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f07e8ac-f965-634e-8000-5e88606453a5'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}, 'thread_id': '2'}, created_at='2025-08-21T12:31:50.180743+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f07e8ac-f962-6c6c-bfff-47e255e280f2'}}, tasks=(PregelTask(id='a190907f-76ec-1e34-adcc-456effa3d6bb', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'actual_joke': "Why did the unpaid intern bring a ladder to the office?  Because they heard the company was looking for someone to climb the corporate ladder... eventually, maybe, if they're lucky."}),), interrupts=())

In [76]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f07e8ac-f965-634e-8000-5e88606453a5"}})

{'topic_of_joke': 'unpaid_internships',
 'actual_joke': "Why did the unpaid intern bring a ladder to the office?  Because they heard the company was looking for someone to climb the corporate ladder... eventually, maybe, if they're lucky.",
 'explanation': 'The humor in the joke lies in the juxtaposition of the literal and figurative meanings of "climbing the corporate ladder."\n\n* **Literal:** The intern literally brings a ladder, a physical object used for climbing. This is a silly, unexpected action.\n\n* **Figurative:** "Climbing the corporate ladder" is an idiom meaning to advance one\'s career within a company, achieving higher positions and responsibilities.\n\nThe joke plays on the intern\'s naivete or desperation.  They misunderstand the idiom, taking it literally.  The added "eventually, maybe, if they\'re lucky" emphasizes their precarious and low-status position within the company, highlighting the long and uncertain path to career advancement, especially for an unpaid int

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student is humorously illustrating their determination to excel in school and eventually land a high-paying job. The image of physically climbing up a ladder mirrors the idea of moving up the academic and professional ranks to achieve success.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-5a0d-6faa-8008-73c3d3425512'}}, metadata={'source': 'loop', 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:06:33.198993+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-4424-6b57-8007-721ec948dc11'}},

#### Updating State

In [ ]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}, metadata={'source': 'update', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:09:21.480601+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0d6e0d3b-0f3c-4ae1-8654-e949524f0ca3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}, metadata={'source': 'update', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:09:21.480601+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0d6e0d3b-0f3c-4ae1-8654-e949524f0ca3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student

### Fault Tolerance

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [ ]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [ ]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [ ]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))